# Synthetic data generation

The goal here is to generate synthetic user sign up locations for my a public pydeck demo.

You must enable pydeck as a notebook extension for this to work. See [the installation instructions](https://pydeck.gl/installation.html).

In [ ]:
import pydeck as pdk

import ipywidgets as widgets
import numpy as np
import pandas as pd 

# Fill color we'll use later for our visualiztion, specified as an RGBA value
FILL_COLOR_RGBA = [155, 0, 250, 80]

# df will store the data we'll generate
df = pd.DataFrame()


## Define pydeck components

# Declare a ScatterplotLayer and render it
layer = pdk.Layer(
    'ScatterplotLayer',
    df,
    get_position=['x', 'y'],
    get_fill_color=FILL_COLOR_RGBA,
    get_radius=500,  # 500 meters of radius
    pickable=True
)
deck = pdk.Deck([layer])

In [ ]:
# Define a function that has no-operation
def noop():
    return 0


## User interface elements

# Create a dropdown that lets the user select what pattern of random points they wish to generate
dropdown = widgets.Dropdown(
    options=[
        ('Cauchy', lambda: np.random.standard_cauchy),
        ('Normal', lambda: np.random.standard_normal),
        ('Uniform', lambda: np.random.uniform),
        ('None', lambda: noop)
    ],
    description='Noise type:',
)


# Create a text input that lets users define the spread of the random points they generate
scaling_factor_input = widgets.FloatText(
    value=1,
    min=0,
    max=30000,
    readout=True,
    description='Scaling:'
)

# Create a slider to let users pick the number of points to render
num_points_slider = widgets.IntSlider(
    value=500,
    min=1,
    max=1000,
    description='# points:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [ ]:
# Define the function that pydeck will execute on click
def add_points(widget_instance, payload):
    # Add the elements we've declared elsewhere
    # in the Notebook for access in this function
    global df
    global d
    global dropdown
    global scaling_factor_input
    global num_points_slider

    # Get the current values associated with our UI elements
    scaling_factor = scaling_factor_input.value
    noise_type = dropdown.value()
    num_points = num_points_slider.value

    try:
        # Pull the current click location out of our map
        x, y = payload['data']['lngLat']
        # Add clicked point to the data we'll export later
        df = df.append({'x': x, 'y': y}, ignore_index=True)
        # Add noise around that center point
        synth = pd.DataFrame([
            {'x': x + noise_type() * scaling_factor,
             'y': y + noise_type() * scaling_factor} for _ in range(0, num_points)])
        # Combine that new data with the data we've already aggregated
        df = pd.concat([df, synth])
        # Render the new data in our deck visualization
        deck.layers[0].data = df  # Modify the data set we're seeing
        deck.update()  # `deck.update()` actually trigger the update
        lng_lat = str(payload['data']['lngLat'])
        # Display update
        text.value = f'Added {num_points} points centered at {lng_lat}, scaled by {scaling_factor}' 
    except Exception as e:
        # Display the error message if there's an issue
        text.value = 'Error: %s' % e

In [ ]:
# Render everything
text = widgets.HTML(value='Synthetic GIS data: Click to create fake points')
display(text)
display(dropdown)
display(scaling_factor_input)
display(num_points_slider)
deck.deck_widget.on_click(add_points)
deck.show()

In [ ]:
df.to_csv('synthetic.csv', index=False)

In [ ]:
layer = pdk.Layer(
    'HexagonLayer',
    df,
    get_position=['x', 'y'],
    elevation_scale=10,
    pickable=True,
    extruded=True
)
view = pdk.data_utils.compute_view(df, 0.6)
deck = pdk.Deck(
    [layer],
    initial_view_state=view
)
deck.show()